In [18]:
#conexion a directorio en Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
#configuraciones iniciales 
directorio = '/content/drive/MyDrive/PT/dataset/img'    #directorio que contiene las imagenes y el archivo de etiquetado en formato json
w_i = 3264    #ancho de imagenes en entero, ej: 3264
h_i = 2448    #alto de imagenes en entero, ej: 2448
carpeta = 'labels_v8'    #carpeta donde se almacenara el nuevo etiquetado generado     

In [20]:
#importacion de bibliotecas necesarias
import json
import os
import errno
from PIL import Image
from PIL.ExifTags import TAGS   

In [21]:
#definicion de variables 
w = w_i
h = h_i    #se asignan valores recibidos de ancho y alto a variables auxiliares 
directorio = directorio.replace('\\', '/') + "/"    #se modifica el directorio para correcta lectura de python
contenido_directorio = os.listdir(directorio)
data = {}
nombre_archivos_jpg = []        
ejpg = '.jpg'
ejson = '.json'
etxt = '.txt'
#archivo_json = 'berries.json'    #nombre del archivo que contiene el etiquetado original

In [22]:
#comprobacion de archivos en directorio
try:
    os.mkdir(directorio + carpeta)    #se crea una carpeta para almacenar todos los archivos de etiquetado creados
except OSError as e:
    if e.errno != errno.EEXIST:
        raise

for archivo in contenido_directorio:
    if os.path.isfile(os.path.join(directorio, archivo)) and archivo.endswith(ejpg):
        nombre_archivos_jpg.append(archivo.replace(ejpg, ''))    #se obtienen los nombres de los archivos JPG encontrados y se almacenan en una lista sin la extension '.jpg'
        print("Archivo JPG encontrado: ", archivo)

    elif os.path.isfile(os.path.join(directorio, archivo)) and archivo.endswith(ejson):
      archivo_json = archivo    #se obtiene el nombre del ultimo archivo JSON encontrado y se almacena en la variable archivo_json
      print("Archivo JSON encontrado: ", archivo)

etiquetas = directorio + archivo_json

Archivo JPG encontrado:  IMG_1004.jpg
Archivo JPG encontrado:  IMG_1007.jpg
Archivo JPG encontrado:  IMG_1008.jpg
Archivo JPG encontrado:  IMG_1009.jpg
Archivo JPG encontrado:  IMG_1010.jpg
Archivo JPG encontrado:  IMG_1011.jpg
Archivo JPG encontrado:  IMG_1012.jpg
Archivo JPG encontrado:  IMG_1013.jpg
Archivo JPG encontrado:  IMG_1014.jpg
Archivo JPG encontrado:  IMG_1015.jpg
Archivo JPG encontrado:  IMG_1016.jpg
Archivo JPG encontrado:  IMG_1017.jpg
Archivo JPG encontrado:  IMG_1018.jpg
Archivo JPG encontrado:  IMG_1019.jpg
Archivo JPG encontrado:  IMG_1020.jpg
Archivo JPG encontrado:  IMG_1021.jpg
Archivo JPG encontrado:  IMG_1022.jpg
Archivo JPG encontrado:  IMG_1023.jpg
Archivo JPG encontrado:  IMG_1024.jpg
Archivo JPG encontrado:  IMG_1025.jpg
Archivo JPG encontrado:  IMG_1026.jpg
Archivo JPG encontrado:  IMG_1027.jpg
Archivo JPG encontrado:  IMG_1028.jpg
Archivo JPG encontrado:  IMG_1029.jpg
Archivo JPG encontrado:  IMG_1030.jpg
Archivo JPG encontrado:  IMG_1031.jpg
Archivo JPG 

In [24]:
#generacion de etiquetas
with open(etiquetas) as file:
    data = json.load(file)

    for nombre in nombre_archivos_jpg:    #para cada nombre almacenado en 'nombre_archivos_jpg', obtenidos a partir del dataset de imagenes
        puntos = []
        lineas = []
        archivo_txt = directorio + carpeta + '/' + nombre + etxt    #se genera un archivo '.txt' para almacenar el etiquetado en formato 'YOLO DARKNET TXT'
        imagen = nombre + ejpg    #imagen es el "objeto" a buscar en el json (nombre de la imagen, cuyo etiquetado esta contenido en el objeto)
        archivo_jpg = directorio + imagen    #ubicacion de cada imagen para lectura de metadatos
        image = Image.open(archivo_jpg)    #Lee la imagen en el directorio señalado
        exifdata = image.getexif() 

        for tagid in exifdata: 
            tagname = TAGS.get(tagid, tagid) 
            value = exifdata.get(tagid) 
            if (tagname == "Orientation" and value == 1):    #si la orientacion de la imagen es horizontal
                w = w_i
                h = h_i
            if (tagname == "Orientation" and value == 6):    #si la orientacion de la imagen es vertical
                w = h_i
                h = w_i
            #print(f"{tagname:25}: {value}") 
        image.close()

        try:    #se usa try-except en caso de que no exista etiquetado en el json para alguna imagen del dataset
            for objeto in data[imagen]:
                puntos.append("0")    #se agrega a la lista un 0 (clase a la que pertenece el etiquetado), y luego todos los puntos del bounding box

                punto_x = int(objeto['x'])    #los puntos obtenidos del archivo json se transforman a formato numerico
                punto_y = int(objeto['y'])
                punto_w = int(objeto['w'])
                punto_h = int(objeto['h'])

                bbox_w = (punto_w - punto_x)/w    #coordenadas normalizadas segun requerimientos de YOLO 
                bbox_h = (punto_h - punto_y)/h
                bbox_x = (punto_x/w + bbox_w/2)
                bbox_y = (punto_y/h + bbox_h/2)

                puntos.append(str("{0:.6f}".format(bbox_x)))    #se limita la cantidad de decimales en las coordenadas normalizadas 
                puntos.append(str("{0:.6f}".format(bbox_y)))    #y se almacenan en una lista como string
                puntos.append(str("{0:.6f}".format(bbox_w))) 
                puntos.append(str("{0:.6f}".format(bbox_h))) 
                
                lineas.append(" ".join(puntos))    #la lista de puntos previa es un bounding box y este se almacenan en otra lista
                puntos = []

        except KeyError:    #se controla excepcion en caso de que no exista etiquetado en json para alguna imagen del dataset
            lineas = ""    #se eliminan los puntos almacenados del archivo anterior
            archivo_txt = ""    #se elimina el nombre del archivo sin etiquetado
            print("Error en: ", imagen)

        if archivo_txt != "":    #no se consideran los nombres de archivo sin etiquetado
            print("======================================")
            print("Imagen: ", archivo_jpg)
            print("Lineas generadas (etiquetas): ")
            with open(archivo_txt, "w") as file:
                for linea in lineas:
                    print(linea)
                    file.write(linea + "\n")    #se escriben los puntos de los bounding box en el archivo '.txt'
            print("Archivo de texto generado: ", archivo_txt)
            print("======================================")

            WINDOWS_LINE_ENDING = b'\r\n'
            UNIX_LINE_ENDING = b'\n'
            with open(archivo_txt, 'rb') as open_file:
                content = open_file.read()
                content = content.replace(WINDOWS_LINE_ENDING, UNIX_LINE_ENDING)    #conversion de formato de endline windows->unix
            with open(archivo_txt, 'wb') as open_file:
                open_file.write(content)

Imagen:  /content/drive/MyDrive/PT/dataset/img/IMG_1004.jpg
Lineas generadas (etiquetas): 
0 0.595741 0.653595 0.060968 0.084967
0 0.700521 0.226716 0.037684 0.046569
0 0.665748 0.206291 0.037377 0.049837
0 0.655484 0.241422 0.033395 0.046569
0 0.683211 0.263685 0.036152 0.046160
0 0.708793 0.257149 0.023591 0.033088
0 0.615196 0.245098 0.025123 0.035131
0 0.597886 0.266340 0.029105 0.040850
0 0.574908 0.490400 0.044424 0.066585
0 0.546415 0.521855 0.043199 0.059232
0 0.441483 0.707312 0.061275 0.081291
0 0.486673 0.709150 0.045650 0.062908
0 0.444240 0.636846 0.058211 0.074346
0 0.299173 0.665645 0.040135 0.053513
0 0.327512 0.735907 0.047181 0.068219
0 0.316636 0.794322 0.049939 0.059232
0 0.257506 0.730392 0.040135 0.049837
0 0.287377 0.401552 0.033088 0.040850
0 0.294884 0.371936 0.029105 0.037173
0 0.313725 0.345180 0.027574 0.035131
0 0.276195 0.316585 0.027880 0.033497
0 0.191483 0.576797 0.047181 0.062908
0 0.197763 0.628472 0.040135 0.044526
0 0.159620 0.635825 0.047181 0.0592